In [1]:
# import argparse
# import importlib
import sys
# import argparse
import numpy as np
import pandas as pd
import datetime as dt
# from dateutil.parser import parse
from Kernel import Kernel
from util import util
from util.order import LimitOrder
from util.oracle.SparseMeanRevertingOracle import SparseMeanRevertingOracle
from tqdm import tqdm
from agent.ExchangeAgent import ExchangeAgent
from agent.NoiseAgent import NoiseAgent
from agent.ValueAgent import ValueAgent
from agent.market_makers.AdaptiveMarketMakerAgent import AdaptiveMarketMakerAgent
from agent.examples.MomentumAgent import MomentumAgent
from agent.examples.ExampleExperimentalAgent import ExampleExperimentalAgentTemplate, ExampleExperimentalAgent
from model.LatencyModel import LatencyModel

In [14]:
#market config
Exchange_Agent = 1
POV_Market_Maker_Agent = 1
Value_Agents = 100
Momentum_Agents = 25
Noise_Agents = 5000
seed = 413
#agent config
ticker = 'ABM'
log_dir = f'log/experimental_agent_demo_short_2min_long_5min_{seed}'
historical_date = 20200603
start_time = dt.datetime.strptime('09:30:00','%H:%M:%S')
end_time = dt.datetime.strptime('10:30:00','%H:%M:%S')

verbose = False
fund_vol = 1e-8
experimental_agent = True
ea_short_window = '2min'
ea_long_window = '5min'

if __name__ == '__main__':

    # Print system banner.
    system_name = "ABIDES: Agent-Based Interactive Discrete Event Simulation"

    print ("=" * len(system_name))
    print (system_name)
    print ("=" * len(system_name))
    print ()

#     rgs, remaining_args = parser.parse_known_args()

    # if config_help:
    #     parser.print_help()
    #     sys.exit()

#     log_dir = log_dir  # Requested log directory.
#     seed = seed  # Random seed specification on the command line.
    if not seed: seed = int(pd.Timestamp.now().timestamp() * 1000000) % (2 ** 32 - 1)
    np.random.seed(seed)

    util.silent_mode = not verbose
    LimitOrder.silent_mode = not verbose

    exchange_log_orders = True
    log_orders = None
    book_freq = 0

    simulation_start_time = dt.datetime.now()
    print("Simulation Start Time: {}".format(simulation_start_time))
    print("Configuration seed: {}\n".format(seed))
    ########################################################################################################################
    ############################################### AGENTS CONFIG ##########################################################

    # Historical date to simulate.
    historical_date = pd.to_datetime(historical_date)
    mkt_open = historical_date + pd.to_timedelta(start_time.strftime('%H:%M:%S'))
    mkt_close = historical_date + pd.to_timedelta(end_time.strftime('%H:%M:%S'))
    agent_count, agents, agent_types = 0, [], []

    # Hyperparameters
    symbol = ticker
    starting_cash = 10000000  # Cash in this simulator is always in CENTS.

    r_bar = 1e5
    sigma_n = r_bar / 10
    kappa = 1.67e-15
    lambda_a = 7e-11

    # Oracle
    symbols = {symbol: {'r_bar': r_bar,
                        'kappa': 1.67e-16,
                        'sigma_s': 0,
                        'fund_vol': fund_vol,
                        'megashock_lambda_a': 2.77778e-18,
                        'megashock_mean': 1e3,
                        'megashock_var': 5e4,
                        'random_state': np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64'))}}

    oracle = SparseMeanRevertingOracle(mkt_open, mkt_close, symbols)

#     # 1) Exchange Agent

#     #  How many orders in the past to store for transacted volume computation
#     # stream_history_length = int(pd.to_timedelta(mm_wake_up_freq).total_seconds() * 100)
#     stream_history_length = 25000

#     agents.extend([ExchangeAgent(id=0,
#                                  name="EXCHANGE_AGENT",
#                                  type="ExchangeAgent",
#                                  mkt_open=mkt_open,
#                                  mkt_close=mkt_close,
#                                  symbols=[symbol],
#                                  log_orders=exchange_log_orders,
#                                  pipeline_delay=0,
#                                  computation_delay=0,
#                                  stream_history=stream_history_length,
#                                  book_freq=book_freq,
#                                  wide_book=True,
#                                  random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64')))])
#     agent_types.extend("ExchangeAgent")
#     agent_count += 1

    # 2) Noise Agents
    num_noise = 5000
    noise_mkt_open = historical_date + pd.to_timedelta("09:00:00")  # These times needed for distribution of arrival times
                                                                    # of Noise Agents
    noise_mkt_close = historical_date + pd.to_timedelta("16:00:00")
    agents.extend([NoiseAgent(id=j,
                              name="NoiseAgent {}".format(j),
                              type="NoiseAgent",
                              symbol=symbol,
                              starting_cash=starting_cash,
                              wakeup_time=util.get_wake_time(noise_mkt_open, noise_mkt_close),
                              log_orders=log_orders,
                              random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64')))
                   for j in range(agent_count, agent_count + num_noise)])
    agent_count += num_noise
    agent_types.extend(['NoiseAgent'])

    # 3) Value Agents
    num_value = 100
    agents.extend([ValueAgent(id=j,
                              name="Value Agent {}".format(j),
                              type="ValueAgent",
                              symbol=symbol,
                              starting_cash=starting_cash,
                              sigma_n=sigma_n,
                              r_bar=r_bar,
                              kappa=kappa,
                              lambda_a=lambda_a,
                              log_orders=log_orders,
                              random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64')))
                   for j in range(agent_count, agent_count + num_value)])
    agent_count += num_value
    agent_types.extend(['ValueAgent'])

    # 4) Market Maker Agents

    """
    window_size ==  Spread of market maker (in ticks) around the mid price
    pov == Percentage of transacted volume seen in previous `mm_wake_up_freq` that
           the market maker places at each level
    num_ticks == Number of levels to place orders in around the spread
    wake_up_freq == How often the market maker wakes up

    """

    # each elem of mm_params is tuple (window_size, pov, num_ticks, wake_up_freq, min_order_size)
    mm_params = [('adaptive', 0.025, 10, '10S', 1),
                 ('adaptive', 0.025, 10, '10S', 1)
                 ]

    num_mm_agents = len(mm_params)
    mm_cancel_limit_delay = 50  # 50 nanoseconds

    agents.extend([AdaptiveMarketMakerAgent(id=j,
                                    name="ADAPTIVE_POV_MARKET_MAKER_AGENT_{}".format(j),
                                    type='AdaptivePOVMarketMakerAgent',
                                    symbol=symbol,
                                    starting_cash=starting_cash,
                                    pov=mm_params[idx][1],
                                    min_order_size=mm_params[idx][4],
                                    window_size=mm_params[idx][0],
                                    num_ticks=mm_params[idx][2],
                                    wake_up_freq=mm_params[idx][3],
                                    cancel_limit_delay=mm_cancel_limit_delay,
                                    skew_beta=0,
                                    level_spacing=5,
                                    spread_alpha=0.75,
                                    backstop_quantity=50000,
                                    log_orders=log_orders,
                                    random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32,
                                                                                              dtype='uint64')))
                   for idx, j in enumerate(range(agent_count, agent_count + num_mm_agents))])
    agent_count += num_mm_agents
    agent_types.extend('POVMarketMakerAgent')


    # 5) Momentum Agents
    num_momentum_agents = 25

    agents.extend([MomentumAgent(id=j,
                                 name="MOMENTUM_AGENT_{}".format(j),
                                 type="MomentumAgent",
                                 symbol=symbol,
                                 starting_cash=starting_cash,
                                 min_size=1,
                                 max_size=10,
                                 wake_up_freq='20s',
                                 log_orders=log_orders,
                                 random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32,
                                                                                           dtype='uint64')))
                   for j in range(agent_count, agent_count + num_momentum_agents)])
    agent_count += num_momentum_agents
    agent_types.extend("MomentumAgent")

    # 6) Experimental Agent

    #### Example Experimental Agent parameters

    if experimental_agent:
        experimental_agent = ExampleExperimentalAgent(
            id=agent_count,
            name='EXAMPLE_EXPERIMENTAL_AGENT',
            type='ExampleExperimentalAgent',
            symbol=symbol,
            starting_cash=starting_cash,
            levels=5,
            subscription_freq=1e9,
            wake_freq='10s',
            order_size=100,
            short_window=ea_short_window,
            long_window=ea_long_window,
            log_orders=True,
            random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64'))
        )
#     else:
#         experimental_agent = ExampleExperimentalAgentTemplate(
#             id=agent_count,
#             name='EXAMPLE_EXPERIMENTAL_AGENT',
#             type='ExampleExperimentalAgent',
#             symbol=symbol,
#             starting_cash=starting_cash,
#             levels=5,
#             subscription_freq=1e9,args, remaining_args = parser.parse_known_args()

    # if config_help:
    #     parser.print_help()
    #     sys.exit()


    experimental_agents = [experimental_agent]
    agents.extend(experimental_agents)
    agent_types.extend("ExperimentalAgent")
    agent_count += 1


    ########################################################################################################################
    ########################################### KERNEL AND OTHER CONFIG ####################################################

    kernel = Kernel("RMSC03 Kernel", random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32,
                                                                                                      dtype='uint64')))

    kernelStartTime = historical_date
    kernelStopTime = mkt_close + pd.to_timedelta('00:01:00')

    defaultComputationDelay = 50  # 50 nanoseconds

    # LATENCY

    latency_rstate = np.random.RandomState(seed=np.random.randint(low=0, high=2**32))
    pairwise = (agent_count, agent_count)

    # All agents sit on line from Seattle to NYC
    nyc_to_seattle_meters = 3866660
    pairwise_distances = util.generate_uniform_random_pairwise_dist_on_line(0.0, nyc_to_seattle_meters, agent_count,
                                                                            random_state=latency_rstate)
    pairwise_latencies = util.meters_to_light_ns(pairwise_distances)

    model_args = {
        'connected': True,
        'min_latency': pairwise_latencies
    }

    latency_model = LatencyModel(latency_model='deterministic',
                                 random_state=latency_rstate,
                                 kwargs=model_args
                                 )
    # KERNEL

    kernel.runner(agents=agents,
                  startTime=kernelStartTime,
                  stopTime=kernelStopTime,
                  agentLatencyModel=latency_model,
                  defaultComputationDelay=defaultComputationDelay,
                  oracle=oracle,
                  log_dir=log_dir)

    simulation_end_time = dt.datetime.now()
    print("Simulation End Time: {}".format(simulation_end_time))
    print("Time taken to run simulation: {}".format(simulation_end_time - simulation_start_time))

ABIDES: Agent-Based Interactive Discrete Event Simulation

Simulation Start Time: 2022-02-22 22:53:44.033625
Configuration seed: 413


--- Simulation time: 1970-01-01 00:00:00.020200603, messages processed: 0, wallclock elapsed: 0 days 00:00:00.000023 ---


--- Simulation time: 1970-01-01 09:33:27.427140065, messages processed: 100000, wallclock elapsed: 0 days 00:00:08.190817 ---


--- Simulation time: 1970-01-01 09:44:01.298631973, messages processed: 200000, wallclock elapsed: 0 days 00:00:17.755338 ---


--- Simulation time: 1970-01-01 09:54:30.140274347, messages processed: 300000, wallclock elapsed: 0 days 00:00:29.775313 ---


--- Simulation time: 1970-01-01 10:04:53.934737076, messages processed: 400000, wallclock elapsed: 0 days 00:00:43.790443 ---


--- Simulation time: 1970-01-01 10:15:22.607996775, messages processed: 500000, wallclock elapsed: 0 days 00:01:00.945079 ---


--- Simulation time: 1970-01-01 10:25:51.936128843, messages processed: 600000, wallclock elapsed: 0 d

Final relative surplus NoiseAgent 1112 0.0001332
Final holdings for NoiseAgent 1113: { ABM: 43, CASH: 5700817 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 1113 -0.4299183
Final holdings for NoiseAgent 1114: { ABM: -31, CASH: 13098543 }.  Marked to market: 10001085
Final relative surplus NoiseAgent 1114 0.3098543
Final holdings for NoiseAgent 1115: { ABM: 23, CASH: 7700437 }.  Marked to market: 9999356
Final relative surplus NoiseAgent 1115 -0.2299563
Final holdings for NoiseAgent 1116: { ABM: 76, CASH: 2400608 }.  Marked to market: 10000760
Final relative surplus NoiseAgent 1116 -0.74993895
Final holdings for NoiseAgent 1117: { CASH: 9999250 }.  Marked to market: 9999250
Final relative surplus NoiseAgent 1117 -7.5e-05
Final holdings for NoiseAgent 1118: { ABM: 39, CASH: 6099766 }.  Marked to market: 9999025
Final relative surplus NoiseAgent 1118 -0.3900234
Final holdings for NoiseAgent 1119: { ABM: 31, CASH: 6901457 }.  Marked to market: 9998915
Final relative surpl

Final relative surplus NoiseAgent 2235 0.19994
Final holdings for NoiseAgent 2236: { ABM: -30, CASH: 13000180 }.  Marked to market: 10002640
Final relative surplus NoiseAgent 2236 0.300018
Final holdings for NoiseAgent 2237: { ABM: -35, CASH: 13497130 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 2237 0.349713
Final holdings for NoiseAgent 2238: { ABM: 30, CASH: 7001410 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 2238 -0.299859
Final holdings for NoiseAgent 2239: { ABM: 47, CASH: 5303854 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 2239 -0.4696146
Final holdings for NoiseAgent 2240: { ABM: -21, CASH: 12099601 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 2240 0.2099601
Final holdings for NoiseAgent 2241: { ABM: -44, CASH: 14396392 }.  Marked to market: 9999032
Final relative surplus NoiseAgent 2241 0.4396392
Final holdings for NoiseAgent 2242: { ABM: -29, CASH: 12899449 }.  Marked to market: 10000000
Final r

Final holdings for NoiseAgent 3333: { ABM: 21, CASH: 7900399 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 3333 -0.2099601
Final holdings for NoiseAgent 3334: { ABM: 46, CASH: 5400874 }.  Marked to market: 9998712
Final relative surplus NoiseAgent 3334 -0.4599126
Final holdings for NoiseAgent 3335: { ABM: -21, CASH: 12099013 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 3335 0.2099013
Final holdings for NoiseAgent 3336: { ABM: -43, CASH: 14299183 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 3336 0.4299183
Final holdings for NoiseAgent 3337: { ABM: -41, CASH: 14098770 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 3337 0.409877
Final holdings for NoiseAgent 3338: { ABM: 43, CASH: 5700817 }.  Marked to market: 9998796
Final relative surplus NoiseAgent 3338 -0.4299183
Final holdings for NoiseAgent 3339: { ABM: -24, CASH: 12398872 }.  Marked to market: 10000840
Final relative surplus NoiseAgent 3339 0.2398872
Final

Final holdings for NoiseAgent 4433: { CASH: 10000000 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 4433 0.0
Final holdings for NoiseAgent 4434: { ABM: 38, CASH: 6200722 }.  Marked to market: 9998936
Final relative surplus NoiseAgent 4434 -0.3799278
Final holdings for NoiseAgent 4435: { ABM: 44, CASH: 5602068 }.  Marked to market: 10000000
Final relative surplus NoiseAgent 4435 -0.4397932
Final holdings for NoiseAgent 4436: { CASH: 9998593 }.  Marked to market: 9998593
Final relative surplus NoiseAgent 4436 -0.0001407
Final holdings for NoiseAgent 4437: { ABM: 38, CASH: 6201786 }.  Marked to market: 10001064
Final relative surplus NoiseAgent 4437 -0.3798214
Final holdings for NoiseAgent 4438: { ABM: 42, CASH: 5799748 }.  Marked to market: 9998950
Final relative surplus NoiseAgent 4438 -0.4200252
Final holdings for NoiseAgent 4439: { CASH: 9999890 }.  Marked to market: 9999890
Final relative surplus NoiseAgent 4439 -1.1e-05
Final holdings for NoiseAgent 4440: { ABM: 23

Logging order book to file...


Processing orderbook log: 100%|██████████| 50724/50724 [00:18<00:00, 2779.39it/s]
/home/ict520c/Documents/ADRL/abides/agent/ExchangeAgent.py:108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.logOrderBookSnapshots(symbol)


Order book logging complete!
Time taken to log the order book: 0:00:22.204833
Order book archival complete.
Event Queue elapsed: 0 days 00:01:31.581785, messages: 640238, messages per second: 6990.9
Mean ending value by agent type:
NoiseAgent: -56
ValueAgent: -11090
AdaptivePOVMarketMakerAgent: 251682
MomentumAgent: -20161
ExampleExperimentalAgent: 1124043
Simulation ending!
Simulation End Time: 2022-02-22 22:55:41.755843
Time taken to run simulation: 0:01:57.722218


In [19]:
df = pd.read_csv('orderbook.csv',index_col=False)
df.columns=['ask1','ask1_v','bid1','bid1_v']
df.astype(np.int64)

,ask1,ask1_v,bid1,bid1_v
0,9999999999,9999999999,99981,39
1,9999999999,9999999999,99981,39
2,9999999999,9999999999,99953,29
3,9999999999,9999999999,99918,21
4,99970,37,99918,21
...,...,...,...,...
36094,99952,124,99946,15
36095,99952,124,99951,41
36096,99946,26,99945,20
36097,99945,10,99944,249


In [20]:
df

,ask1,ask1_v,bid1,bid1_v
0,1.000000e+10,1.000000e+10,99981.0,39.0
1,1.000000e+10,1.000000e+10,99981.0,39.0
2,1.000000e+10,1.000000e+10,99953.0,29.0
3,1.000000e+10,1.000000e+10,99918.0,21.0
4,9.997000e+04,3.700000e+01,99918.0,21.0
...,...,...,...,...
36094,9.995200e+04,1.240000e+02,99946.0,15.0
36095,9.995200e+04,1.240000e+02,99951.0,41.0
36096,9.994600e+04,2.600000e+01,99945.0,20.0
36097,9.994500e+04,1.000000e+01,99944.0,249.0


In [2]:
from util.plotting.liquidity_telemetry import create_orderbooks

In [3]:
processed_orderbook, transacted_orders, cleaned_orderbook = create_orderbooks('log/log/experimental_agent_demo_short_2min_long_5min_413/EXCHANGE_AGENT.bz2', 'log/log/experimental_agent_demo_short_2min_long_5min_413/ORDERBOOK_ABM_FULL.bz2')

Constructing orderbook...


Processing order book: 100%|██████████| 36100/36100 [00:05<00:00, 6324.91it/s]


Orderbook construction complete!


/home/ict520c/Documents/ADRL/abides/util/plotting/liquidity_telemetry.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transacted_orders['SIZE'] = transacted_orders['SIZE'] / 2


In [4]:
cleaned_orderbook

,ORDER_ID,PRICE,SIZE,BUY_SELL_FLAG,TYPE,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,MID_PRICE,SPREAD,ORDER_VOLUME_IMBALANCE,VWAP
1970-01-01 09:30:00.020425499,8,999.70,37,0,LIMIT_ORDER,99970.0,37.0,99918.0,21.0,1.000000e+10,1.000000e+10,-1.000000e+10,-1.000000e+10,999.440,0.52,0.637931,999.383563
1970-01-01 09:30:00.020700299,46,998.17,34,1,LIMIT_ORDER,99918.0,18.0,99817.0,34.0,9.997000e+04,3.700000e+01,-1.000000e+10,-1.000000e+10,998.675,1.01,0.346154,999.343981
1970-01-01 09:30:00.020703206,47,999.81,45,0,LIMIT_ORDER,99918.0,18.0,99817.0,34.0,9.997000e+04,3.700000e+01,-1.000000e+10,-1.000000e+10,998.675,1.01,0.346154,999.343981
1970-01-01 09:30:00.020707432,48,999.81,36,0,LIMIT_ORDER,99918.0,18.0,99817.0,34.0,9.997000e+04,3.700000e+01,-1.000000e+10,-1.000000e+10,998.675,1.01,0.346154,999.343981
1970-01-01 09:30:00.020713628,49,999.81,30,0,LIMIT_ORDER,99918.0,18.0,99817.0,34.0,9.997000e+04,3.700000e+01,-1.000000e+10,-1.000000e+10,998.675,1.01,0.346154,999.343981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 10:29:59.708801583,51414,999.46,15,True,ORDER_EXECUTED,99946.0,26.0,99945.0,20.0,9.995200e+04,1.240000e+02,9.994400e+04,2.490000e+02,999.455,0.01,0.565217,999.377699
1970-01-01 10:29:59.898988886,51419,999.45,30,0,LIMIT_ORDER,99945.0,10.0,99944.0,249.0,9.994600e+04,2.600000e+01,9.994200e+04,1.210000e+02,999.445,0.01,0.038610,999.377699
1970-01-01 10:29:59.898988886,51419,999.45,20,0,ORDER_EXECUTED,99945.0,10.0,99944.0,249.0,9.994600e+04,2.600000e+01,9.994200e+04,1.210000e+02,999.445,0.01,0.038610,999.377700
1970-01-01 10:29:59.898988886,51409,999.45,20,True,ORDER_EXECUTED,99945.0,10.0,99944.0,249.0,9.994600e+04,2.600000e+01,9.994200e+04,1.210000e+02,999.445,0.01,0.038610,999.377701


In [5]:
transacted_orders

,ORDER_ID,PRICE,SIZE,BUY_SELL_FLAG,TYPE,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,MID_PRICE,SPREAD,ORDER_VOLUME_IMBALANCE,VWAP
1970-01-01 09:30:00.020755341,9,999.18,9.0,0,ORDER_EXECUTED,99970.0,34.0,99817.0,34.0,99981.0,1377.0,-1.000000e+10,-1.000000e+10,998.935,1.53,0.500000,999.331368
1970-01-01 09:30:00.020755341,52,1000.00,9.0,1,ORDER_EXECUTED,99970.0,34.0,99817.0,34.0,99981.0,1377.0,-1.000000e+10,-1.000000e+10,998.935,1.53,0.500000,999.379127
1970-01-01 09:30:00.020755341,8,999.70,1.5,0,ORDER_EXECUTED,99970.0,34.0,99817.0,34.0,99981.0,1377.0,-1.000000e+10,-1.000000e+10,998.935,1.53,0.500000,999.382902
1970-01-01 09:30:00.020755341,52,1000.00,1.5,1,ORDER_EXECUTED,99970.0,34.0,99817.0,34.0,99981.0,1377.0,-1.000000e+10,-1.000000e+10,998.935,1.53,0.500000,999.390078
1970-01-01 09:30:00.020884282,71,999.70,10.5,1,ORDER_EXECUTED,99918.0,96.0,99896.0,35.0,99953.0,157.0,9.981700e+04,3.400000e+01,999.070,0.22,0.732824,999.413405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 10:29:59.156436327,51414,999.46,15.0,True,ORDER_EXECUTED,99952.0,124.0,99946.0,15.0,99959.0,124.0,9.994500e+04,2.000000e+01,999.490,0.06,0.892086,999.377697
1970-01-01 10:29:59.708801583,51418,999.46,7.5,False,ORDER_EXECUTED,99946.0,26.0,99945.0,20.0,99952.0,124.0,9.994400e+04,2.490000e+02,999.455,0.01,0.565217,999.377698
1970-01-01 10:29:59.708801583,51414,999.46,7.5,True,ORDER_EXECUTED,99946.0,26.0,99945.0,20.0,99952.0,124.0,9.994400e+04,2.490000e+02,999.455,0.01,0.565217,999.377699
1970-01-01 10:29:59.898988886,51419,999.45,10.0,0,ORDER_EXECUTED,99945.0,10.0,99944.0,249.0,99946.0,26.0,9.994200e+04,1.210000e+02,999.445,0.01,0.038610,999.377700


In [24]:
import gym
from gym import spaces
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

class CustomEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self,):
        super(CustomEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(3)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=2, shape=(1, ), dtype=np.float64)
        self.idx = 0
        self.hold_price = -1
        self.price = transacted_orders['PRICE'].to_numpy()
    def step(self, action):
        reward = 0
        if action == 0: #sell
            if self.hold_price != -1:
                reward = self.price[self.idx] - self.hold_price
                self.hold_price = -1
        if action == 2:
            if self.hold_price != -1:
                self.hold_price = self.hold_price + self.price[self.idx] / 2
            else:
                self.hold_price = self.price[self.idx]
                
                
        observation = self.price[self.idx]
        
        self.idx += 1
        done = (self.idx == len(self.price))
        
        return observation, reward, done, {}
    def reset(self):
        self.idx = 0
        observation = self.price[self.idx]
        return observation  # reward, done, info can't be included
    def render(self, mode='human'):
        ...
    def close (self):
        ...

In [25]:
env = CustomEnv()
# Define and Train the agent
model = A2C('MlpPolicy', env).learn(total_timesteps=120000)

In [31]:
import torch
import torch as th
import torch.nn as nn

class Mlp(nn.Module):  

  def __init__(self, n_inputs=1, n_actions=3):
      nn.Module.__init__(self)

      self.fc1 = nn.Linear(n_inputs, 64)
      self.fc2 = nn.Linear(64, 64)      
      self.fc3 = nn.Linear(64, n_actions)      
      self.activ_fn = nn.Tanh()
      self.out_activ = nn.Softmax(dim=0)

  def forward(self, x):
      x = self.activ_fn(self.fc1(x))
      x = self.activ_fn(self.fc2(x))
      x = self.out_activ(self.fc3(x))
      return x


def copy_mlp_weights(baselines_model):
  torch_mlp = Mlp(n_inputs=4, n_actions=2)
  model_params = baselines_model.get_parameters()
  
  policy_keys = [key for key in model_params.keys() if "pi" in key]
  policy_params = [model_params[key] for key in policy_keys]
    
  for (th_key, pytorch_param), key, policy_param in zip(torch_mlp.named_parameters(), policy_keys, policy_params):
    param = th.from_numpy(policy_param)    
    #Copies parameters from baselines model to pytorch model
    print(th_key, key)
    print(pytorch_param.shape, param.shape, policy_param.shape)
    pytorch_param.data.copy_(param.data.clone().t())
    
  return torch_mlp

In [39]:
th_model = copy_mlp_weights(model)
# device = torch.device("cuda" if True else "cpu")
# th_model.to(device)
device = 'cpu'

## Adversarial Attack

In [36]:
def fgsm_attack(model, nn.Softmax(dim=0), price_vec, actions, eps=) :
    
    images = images.to(device)
    labels = labels.to(device)
    price_vec.requires_grad = True
            
    outputs = th_model(price_vec)
    
    model.zero_grad()
    cost = loss(outputs, labels).to(device)
    cost.backward()
    
    attack_images = price_vec + eps*images.grad.sign()
    attack_images = torch.clamp(attack_images, 0, 1)
    
    return attack_prices

__main__.Mlp